In [284]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [285]:
churn = pd.read_csv('./DATA_Customer-Churn.txt')
churn.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class. Here is the list of steps to be followed (building a simple model without balancing the data):

Round 1

 Import the required libraries and modules that you would need.
Read that data into Python and call the dataframe churnData.
Check the datatypes of all the columns in the data.

You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.


Check for null values in the dataframe. Replace the null values.

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

Split the data into a training set and a test set.
Scale the features either by using MinMaxScaler or a standard scaler.
(Optional) Encode the categorical variables so you can use them for modeling later.

In [286]:
churn['TotalCharges'] = pd.to_numeric(churn['TotalCharges'], errors='coerce')


In [287]:
churn.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [288]:
churn.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [289]:
churn = churn.dropna()

In [290]:
churn_data = churn[['tenure','SeniorCitizen','MonthlyCharges','TotalCharges','Churn']]

In [291]:
map_y_n = { "No":0,"Yes":1}

churn_data["Churn"] = churn_data['Churn'].map(map_y_n)
churn_data["Churn"] = pd.to_numeric(churn_data['Churn'], errors='coerce')

In [292]:
y = churn_data['Churn']
X = churn_data.drop('Churn', axis = 1)

In [293]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [294]:
from sklearn.preprocessing import StandardScaler
def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled =pd.DataFrame(X_train_scaled, columns=X_train.columns)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled=pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    return X_train_scaled, X_test_scaled

X_train_scaled, X_test_scaled=scale_features(X_train, X_test)

Round 2

(Optional) Fit a logistic Regression model on the training data.
Fit a Knn Classifier (NOT KnnRegressor please!)model on the training data.
Fit a Decision Tree Classifier on the training data.
Compare the accuracy, precision, recall for the previous models on both the train and test sets.

In [295]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [296]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [297]:
import numpy as np

In [298]:
import warnings
warnings.filterwarnings('ignore')

In [299]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [300]:
    # accuracy_train = accuracy_score(y_train, y_pred_train)
    # precision_train = precision_score(y_train, y_pred_train)
    # recall_train = recall_score(y_train, y_pred_train)

    # accuracy_test = accuracy_score(y_test, y_pred_test)
    # precision_test = precision_score(y_test, y_pred_test)
    # recall_test = recall_score(y_test, y_pred_test)

In [301]:
log_reg = LogisticRegression()
knn_c = KNeighborsClassifier()
dt_c = DecisionTreeClassifier()

model_pipeline = [log_reg, knn_c, dt_c]
model_names = ['Logistic Regression', 'KNN Classifier', 'Decision Tree Classifier']

In [302]:
def evaluation(X_train, X_test, y_train, y_test, k_fold):
    i = 0
    
    for model in model_pipeline:

        model_f = model.fit(X_train_scaled, y_train)
        y_pred_test = model_f.predict(X_test_scaled)
        y_pred_train= model_f.predict(X_train_scaled)

        ave_of_cval_scores = cross_val_score(model, X_train, y_train, cv=k_fold).mean()
        std_of_cval_scores = cross_val_score(model, X_train, y_train, cv=k_fold).std()


        performance_df = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall', 'Ave of Scores', 'Std of Scores'],
                            'Train': [accuracy_score(y_train, y_pred_train),
                                        precision_score(y_train, y_pred_train),
                                        recall_score(y_train, y_pred_train)," "," "],
                            'Test': [accuracy_score(y_test, y_pred_test),
                                    precision_score(y_test, y_pred_test),
                                    recall_score(y_test, y_pred_test)," "," "],
                            'Cros Validation':[' ',' ',' ', ave_of_cval_scores, std_of_cval_scores]
                                    })
        
        pd.options.display.float_format = '{:.2f}'.format
        heads = [[model_names[i],model_names[i],model_names[i],model_names[i]],[' ','Train', 'Test', 'Cross Val']]
        performance_df.columns = heads

        if i > len(model_pipeline)-1:
            continue
        else:
            i += 1
        
        display(performance_df)
    return

In [303]:
evaluation(X_train_scaled, X_test_scaled, y_train, y_test, 5)

Logistic Regression                     
                      Train Test Cross Val
0            Accuracy  0.79 0.78          
1           Precision  0.66 0.62          
2              Recall  0.46 0.44          
3       Ave of Scores                 0.79
4       Std of Scores                 0.01

KNN Classifier                     
                 Train Test Cross Val
0       Accuracy  0.84 0.76          
1      Precision  0.73 0.57          
2         Recall  0.61 0.45          
3  Ave of Scores                 0.77
4  Std of Scores                 0.01

Decision Tree Classifier                     
                           Train Test Cross Val
0                 Accuracy  0.99 0.73          
1                Precision  0.99 0.50          
2                   Recall  0.97 0.48          
3            Ave of Scores                 0.72
4            Std of Scores                 0.00

Round 3

apply K-fold cross validation on your models built before,  and check the model score. Note: So far we have not balanced the data.

In [304]:
# as above

Round 4

fit a Random forest Classifier on the data and compare the accuracy.
tune the hyper parameters with Gridsearch and check the results. retrain the final mode with the best parameters found.

In [305]:
from sklearn.ensemble import RandomForestClassifier

In [306]:
from sklearn.model_selection import GridSearchCV

In [307]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

In [308]:
param_grid = {
    'n_estimators': [50, 100,500],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
    ##'max_samples' : ['None', 0.5],
    ##'max_depth':[3,5,10],
    ## 'bootstrap':[True,False]
    }
random_forest_c= RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(random_forest_c, param_grid, cv=5,return_train_score=True,n_jobs=-1)

In [309]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 500]},
             return_train_score=True)

In [310]:
best_params = grid_search.best_params_
best_params

{'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 500}

In [311]:
rfc_ops = {'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 500}

random_forest_c= RandomForestClassifier(**rfc_ops)
random_forest_c.fit(X_train_scaled, y_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=500)

In [312]:
model_pipeline = [random_forest_c]
model_names = ['Random Forest Classifier']

evaluation(X_train_scaled, X_test_scaled, y_train, y_test, 5)

Random Forest Classifier                     
                           Train Test Cross Val
0                 Accuracy  0.94 0.77          
1                Precision  0.94 0.59          
2                   Recall  0.82 0.44          
3            Ave of Scores                 0.78
4            Std of Scores                 0.01

In [313]:
# -Break-

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and check the accuracy of the model.

In [314]:
churn_data['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [315]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y, test_size=0.2, random_state=42)

# X, y on churn_data dataframe (which has reduced columns from )


In [316]:
churn_train = pd.concat([X_train_d, y_train_d], axis = 1, ignore_index=True)
churn_train.columns = churn_data.columns
churn_train['Churn'].value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [317]:
X_train_d_scaled, X_test_d_scaled=scale_features(X_train_d, X_test_d)

In [330]:
y_test_d.describe()

count   1407.00
mean       0.27
std        0.44
min        0.00
25%        0.00
50%        0.00
75%        1.00
max        1.00
Name: Churn, dtype: float64

* Downsampling & test model

In [318]:
category_0 = churn_train[churn_train['Churn'] == 0] # negative class (majority)
category_1 = churn_train[churn_train['Churn'] == 1]
c0_len = category_0.shape[0] 
c1_len = category_1.shape[0]

print(c0_len, c1_len)

4130 1495


* Scalling down C0 from 4130 to 1495

In [319]:
category_0_down = category_0.sample(c1_len)

In [320]:
data = pd.concat([category_0_down, category_1], axis=0)
data = data.sample(frac=1)
data['Churn'].value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [321]:
y = data['Churn']
X = data.drop('Churn', axis = 1)

In [327]:
X_train.describe()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
count,2990.00,2990.00,2990.00,2990.00
mean,27.70,0.19,67.87,2036.25
std,23.95,0.39,28.97,2177.04
min,1.00,0.00,18.25,18.85
25%,5.00,0.00,45.46,270.16
50%,21.50,0.00,74.62,1136.38
75%,49.00,0.00,91.09,3308.65
max,72.00,1.00,117.80,8684.80


In [323]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y, test_size=0.2, random_state=42)

In [331]:
param_grid = {
    'n_estimators': [50, 100,500],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
    ##'max_samples' : ['None', 0.5],
    ##'max_depth':[3,5,10],
    ## 'bootstrap':[True,False]
    }
random_forest_c= RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(random_forest_c, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train_d, y_train_d)
best_params = grid_search.best_params_


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 500]},
             return_train_score=True)

In [333]:
random_forest_c= RandomForestClassifier(**best_params)
random_forest_c.fit(X_train_d, y_train_d)

model_pipeline = [random_forest_c]
model_names = ['Random Forest Classifier']

evaluation(X_train_d, X_test_d, y_train_d, y_test_d, 5)

RandomForestClassifier(min_samples_leaf=2, n_estimators=500)

Random Forest Classifier                     
                           Train Test Cross Val
0                 Accuracy  0.94 0.77          
1                Precision  0.94 0.60          
2                   Recall  0.81 0.44          
3            Ave of Scores                 0.77
4            Std of Scores                 0.01

In [334]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

y = churn_data['Churn']
X = churn_data.drop('Churn', axis = 1)

X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

1    5163
0    5163
Name: Churn, dtype: int64

In [335]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm, test_size=0.2, random_state=42)

In [336]:
param_grid = {
    'n_estimators': [50, 100,500],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
    ##'max_samples' : ['None', 0.5],
    ##'max_depth':[3,5,10],
    ## 'bootstrap':[True,False]
    }
random_forest_c= RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(random_forest_c, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train_sm, y_train_sm)
best_params = grid_search.best_params_

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 500]},
             return_train_score=True)

In [338]:
def evaluation(X_train, X_test, y_train, y_test, k_fold):
    i = 0
    
    for model in model_pipeline:

        model_f = model.fit(X_train, y_train)
        y_pred_test = model_f.predict(X_test)
        y_pred_train= model_f.predict(X_train)

        ave_of_cval_scores = cross_val_score(model, X_train, y_train, cv=k_fold).mean()
        std_of_cval_scores = cross_val_score(model, X_train, y_train, cv=k_fold).std()


        performance_df = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall', 'Ave of Scores', 'Std of Scores'],
                            'Train': [accuracy_score(y_train, y_pred_train),
                                        precision_score(y_train, y_pred_train),
                                        recall_score(y_train, y_pred_train)," "," "],
                            'Test': [accuracy_score(y_test, y_pred_test),
                                    precision_score(y_test, y_pred_test),
                                    recall_score(y_test, y_pred_test)," "," "],
                            'Cros Validation':[' ',' ',' ', ave_of_cval_scores, std_of_cval_scores]
                                    })
        
        pd.options.display.float_format = '{:.2f}'.format
        heads = [[model_names[i],model_names[i],model_names[i],model_names[i]],[' ','Train', 'Test', 'Cross Val']]
        performance_df.columns = heads

        if i > len(model_pipeline)-1:
            continue
        else:
            i += 1
        
        display(performance_df)
    return

In [339]:
random_forest_c= RandomForestClassifier(**best_params)
random_forest_c.fit(X_train_sm, y_train_sm)

model_pipeline = [random_forest_c]
model_names = ['Random Forest Classifier']

evaluation(X_train_sm, X_test_sm, y_train_sm, y_test_sm, 5)

RandomForestClassifier(min_samples_leaf=2, n_estimators=500)

Random Forest Classifier                     
                           Train Test Cross Val
0                 Accuracy  0.95 0.77          
1                Precision  0.95 0.76          
2                   Recall  0.96 0.78          
3            Ave of Scores                 0.78
4            Std of Scores                 0.01